### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 08 декабря 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 08 декабря, -4 балла после 08:30 15 декабря, -6 баллов после 08:30 22 декабря, -8 баллов после 08:30 29 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, p):
        return 1 - (p**2).sum(axis=0)

    def __entropy(self, p):
        return 1 - np.sum(p*np.log(p), axis=0)

    def __misclass(self, p):
        return 1 - p.max(axis=0)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[0:np.sqrt(n_feature).astype('int')]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[0:np.log2(n_feature).astype('int') + 1]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y, freq):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        l_uniform_classes = np.where(sorted_y[:-1] != sorted_y[1:])[0] + 1
        l_sizes = l_uniform_classes
        l_sizes[1:] -= l_uniform_classes[:-1]
        l_freq = np.zeros(shape=(self.num_class, l_uniform_classes.size))
        l_freq[sorted_y[l_uniform_classes - 1],
               range(l_uniform_classes.size)] = 1
        l_freq = (l_freq * l_sizes).cumsum(axis=1)
        r_freq = freq.reshape(-1, 1) - l_freq
        r_uniform_classes = y.size - l_uniform_classes
        crit = (l_uniform_classes * self.G_function(l_freq / l_uniform_classes) +
                r_uniform_classes * self.G_function(r_freq / r_uniform_classes))

        best_split = crit.argmax()
        pos = l_uniform_classes[best_split]

        return crit[best_split], (sorted_x[pos] + sorted_x[pos - 1]) / 2

    def __fit_node(self, x, y, node_id, depth):
        freq = np.bincount(y, minlength=self.num_class)
        most_common = freq.argmax()
        if (depth == self.max_depth or
            y.size <= self.min_samples_split or
                freq[most_common] / y.size >= self.sufficient_share):
            self.tree[node_id] = (self.LEAF_TYPE,
                                  most_common, freq / y.size)
        else:
            best_crit = np.inf
            best_feature_id = -1
            best_th = 0
            for i in self.get_feature_ids(x.shape[1]):
                crit, th = self.__find_threshold(x[:, i], y, freq.flatten())
                if best_crit > crit:
                    best_crit = crit
                    best_feature_id = i
                    best_th = th
            xl, xr, yl, yr = self.__div_samples(x, y, best_feature_id, best_th)
            if xl.size == 0 or xr.size == 0:
                self.tree[node_id] = (self.LEAF_TYPE, most_common,
                                      freq[most_common] / y.size)
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE,
                                      best_feature_id, best_th)
#                 print(freq)
#                 print(y.size * self.G_function(freq.reshape(-1, 1)), best_crit)
                self.feature_importances_[best_feature_id] += (
                    y.size * self.G_function(freq.reshape(-1, 1) / y.size) -
                    best_crit) / self.total
                self.__fit_node(xl, yl, 2 * node_id + 1, depth + 1)
                self.__fit_node(xr, yr, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1], dtype=np.float)
        self.total = y.size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [5]:
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [6]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.9444444444444444
1.0


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [7]:
%%time
clf.fit(X_train, y_train)

CPU times: user 3.39 ms, sys: 0 ns, total: 3.39 ms
Wall time: 2.33 ms


DecisionTreeClassifier()

In [8]:
%%time 
my_clf.fit(X_train, y_train)

CPU times: user 12.3 ms, sys: 137 µs, total: 12.5 ms
Wall time: 14.1 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [9]:
df = pd.read_csv('data/Speed Dating Data.csv'
                 , sep=',', encoding="ISO-8859-1")

In [10]:
df

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,22.0,1,44,2,21,22,14,10.0,5,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8374,552,22.0,1,44,2,21,22,13,10.0,4,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8375,552,22.0,1,44,2,21,22,19,10.0,10,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8376,552,22.0,1,44,2,21,22,3,10.0,16,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0


Данные предоставляют огромное колличество признаков, но так ли они нужны на самом деле? У участников есть только 4 минуты чтобы пообщаться с незнокомым человеком. Их вряд ли интерисует сколько они платят за обучение или какой результат у них был на тестировании. Мне кажется, наиболее важными признаками будут оценки партнеров друг друга (привлекательность, интерес, честность, чувство юмора, амбиции). Конечно другие признаки могут давать нам интересные скрытые зависимости, поэтому я поступлю следующим образом. Построим базовую модель, которая будет включать себя лишь оценки портнеров, далее подумаем что можно добавить еще.

In [11]:
df_base = df[['match', 'iid', 'pid', 'gender', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'int_corr']]

In [12]:
df_base

,match,iid,pid,gender,attr_o,sinc_o,intel_o,fun_o,amb_o,int_corr
0,0,1,11.0,0,6.0,8.0,8.0,8.0,8.0,0.14
1,0,1,12.0,0,7.0,8.0,10.0,7.0,7.0,0.54
2,1,1,13.0,0,10.0,10.0,10.0,10.0,10.0,0.16
3,1,1,14.0,0,7.0,8.0,9.0,8.0,9.0,0.61
4,1,1,15.0,0,8.0,7.0,9.0,6.0,9.0,0.21
...,...,...,...,...,...,...,...,...,...,...
8373,0,552,526.0,1,10.0,5.0,3.0,2.0,6.0,0.64
8374,0,552,527.0,1,6.0,3.0,7.0,3.0,7.0,0.71
8375,0,552,528.0,1,2.0,1.0,2.0,2.0,2.0,-0.46
8376,0,552,529.0,1,5.0,7.0,5.0,5.0,3.0,0.62


In [13]:
df_base.drop_duplicates(['iid']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551 entries, 0 to 8356
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   match     551 non-null    int64  
 1   iid       551 non-null    int64  
 2   pid       551 non-null    float64
 3   gender    551 non-null    int64  
 4   attr_o    543 non-null    float64
 5   sinc_o    529 non-null    float64
 6   intel_o   538 non-null    float64
 7   fun_o     536 non-null    float64
 8   amb_o     513 non-null    float64
 9   int_corr  544 non-null    float64
dtypes: float64(7), int64(3)
memory usage: 47.4 KB


Нуллов не так много, я думаю можно их просто выкинуть

In [14]:
df_base = df_base.dropna()

In [15]:
df_male = df_base.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender', 'match', 'int_corr'], axis=1)\
                                 .dropna()
df_female = df_base.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender'], axis=1)\
                                   .dropna()
df_male.columns = df_male.columns + '_m'        
df_female.columns = ['match'] + list(df_female.columns[1:-1] + '_f') + ['int_corr']

In [16]:
df_pair = pd.merge(df_female, df_male, left_on='pid_f', right_on='iid_m',how='inner')
df_pair = df_pair.drop(['iid_m', 'iid_f', 'pid_f', 'pid_m'], axis=1)

In [17]:
df_pair

,match,attr_o_f,sinc_o_f,intel_o_f,fun_o_f,amb_o_f,int_corr,attr_o_m,sinc_o_m,intel_o_m,fun_o_m,amb_o_m
0,0,6.0,8.0,8.0,8.0,8.0,0.14,6.0,9.0,7.0,7.0,6.0
1,0,6.0,8.0,8.0,8.0,8.0,0.14,5.0,7.0,8.0,4.0,6.0
2,0,6.0,8.0,8.0,8.0,8.0,0.14,7.0,9.0,10.0,7.0,8.0
3,0,6.0,8.0,8.0,8.0,8.0,0.14,4.0,10.0,8.0,5.0,8.0
4,0,6.0,8.0,8.0,8.0,8.0,0.14,5.0,8.0,8.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3787,0,5.0,10.0,10.0,5.0,5.0,0.35,4.0,3.0,9.0,4.0,9.0
3788,0,5.0,10.0,10.0,5.0,5.0,0.35,4.0,8.0,9.0,3.0,8.0
3789,0,5.0,10.0,10.0,5.0,5.0,0.35,1.0,5.0,8.0,1.0,8.0
3790,0,5.0,10.0,10.0,5.0,5.0,0.35,4.0,6.0,5.0,5.0,6.0


In [18]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier()
clf = DecisionTreeClassifier()

In [31]:
%time clf.fit(X_train,y_train)

%time my_clf.fit(X_train, y_train)

CPU times: user 7.66 ms, sys: 3.89 ms, total: 11.5 ms
Wall time: 10.9 ms
CPU times: user 68.3 ms, sys: 229 µs, total: 68.6 ms
Wall time: 72.3 ms


In [32]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9829382183908046

In [33]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8884470657308319

Результат просто отличный, мне кажется можно на этом остановиться.

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [34]:
dict_imp = dict(zip(df_pair.columns[1:], clf.feature_importances_))
my_dict_imp = dict(zip(df_pair.columns[1:], my_clf.feature_importances_))


In [35]:
from operator import itemgetter
sorted(dict_imp.items(), key=itemgetter(1), reverse=True)[:10]

[('int_corr', 0.28697470313700607),
 ('attr_o_f', 0.22698312944355223),
 ('fun_o_f', 0.21666375474318028),
 ('intel_o_f', 0.1660098678790484),
 ('sinc_o_f', 0.0367984635391272),
 ('amb_o_f', 0.03553808878315972),
 ('sinc_o_m', 0.015402570941282065),
 ('amb_o_m', 0.009747155990989097),
 ('fun_o_m', 0.005882265542654814),
 ('attr_o_m', 0.0)]

In [36]:
sorted(my_dict_imp.items(), key=itemgetter(1), reverse=True)[:10]

[('int_corr', 0.044662969243926674),
 ('sinc_o_f', 0.015366475165356924),
 ('intel_o_f', 0.013147848593714415),
 ('attr_o_f', 0.008874703963935363),
 ('fun_o_f', 0.008438035357348836),
 ('amb_o_f', 0.007272214869047179),
 ('fun_o_m', 0.0004470682372206389),
 ('attr_o_m', 0.00023531456893974463),
 ('sinc_o_m', 0.0),
 ('intel_o_m', 0.0)]

Самым важным признаком оказалась корреляция интересов партнеров. Так же интересно заметить что на match в первую очередь влияет то, насколько мужчина понравился женщине, а только потом насколько женщина понравилась мужчине =)

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ



Хорошая дз, как всегда. Однако попытка сравняться со скоростью модели из sklearn мне кажется не очень нужна. Она занимает много времени (я написал два различных решения на numpy которые работали за разное время и не совсем понятно по чему). Мне кажется лучше просто понять как устроен алгоритм, и проверить что по качеству он не уступает модели из коробки. А больше времени как раз уделить применению этой модели на разных данных.